# Tensorboard with Hyperparameter

In [1]:
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.callbacks import TensorBoard

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [3]:
def create_model(param):
    cell_num  = param['cell_num']
    layer_num = param['layer_num']

    inputs = tf.keras.Input(shape=(4,))
    x      = inputs
    
    for _ in range(layer_num):
        x = layers.Dense(cell_num, activation='relu')(x)

    outputs = layers.Dense(3,  activation='softmax')(x)
    model   = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), \
                  loss=tf.keras.losses.sparse_categorical_crossentropy, \
                  metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    return model

In [6]:
from itertools import product
# dict 데이터 입력받아 Cartersian Product 연산 Iterator 리턴
def dict_product(dicts): 
    return (dict(zip(dicts, x)) for x in product(*dicts.values()))

import time as pytime # 시간 모듈 
params = {'cell_num':[8, 16], 'layer_num':[1, 2]}
for param in dict_product(params): # 파라미터 순회 
    tensorboard_name = "iris_param: {}".format(str(param)+" on "+str(int(pytime.time())))
    tensorboard = TensorBoard(log_dir='./logs/{}'.format(tensorboard_name))
    model = create_model(param)
    model.fit(X, Y, epochs=20, callbacks=[tensorboard])

array([0.18146984, 0.81473937, 0.00145312, 0.00233767])

In [8]:
from sklearn.model_selection import KFold

kf = KFold(random_state=30,
           n_splits=10,
           shuffle=True,
          )

In [9]:
# define grid_search
grid_search = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=kf, 
                           n_jobs=-1, 
                           verbose=2
                          )

# fit with (x_train, y_train)
grid_search.fit(X, y)

Fitting 10 folds for each of 96 candidates, totalling 960 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:   39.9s finished


GridSearchCV(cv=KFold(n_splits=10, random_state=30, shuffle=True),
             estimator=RandomForestRegressor(max_depth=2, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [None, 6, 9, 12],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_split': [0.01, 0.05, 0.1],
                         'n_estimators': [100, 150, 200, 250]},
             verbose=2)

In [10]:
grid_search.best_params_

{'max_depth': 9,
 'max_features': 'auto',
 'min_samples_split': 0.01,
 'n_estimators': 200}

In [11]:
estimator = RandomForestRegressor(n_estimators=200,max_depth=9, max_features= 'auto', min_samples_split=0.01, random_state=0)
estimator.fit(X, y)
print(estimator.predict([[0, 0, 0, 0]]))

[-0.21872506]
